<a href="https://colab.research.google.com/github/francji1/01RAD/blob/main/code/01RAD_Ex03_solution_Belo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD Exercise 03

Last exercise: simple linear regression + different approaches how to add categorical varaible

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.datasets import get_rdataset
from scipy.stats import t,norm

from sklearn.linear_model import LinearRegression

# Individual student work

# **Exercise: Developing a Marketing Plan Based on Advertising Data**

Imagine that you are statistical consultants tasked with building a marketing plan for the next year to maximize product sales. You have access to a dataset that contains information on the advertising budget allocated to three different media channels—**TV**, **Radio**, and **Newspaper**—and the corresponding **Sales** figures.

## **Dataset Description**

- **Variables:**
  - **TV**: Advertising budget allocated to TV (in thousands of dollars)
  - **Radio**: Advertising budget allocated to Radio (in thousands of dollars)
  - **Newspaper**: Advertising budget allocated to Newspaper (in thousands of dollars)
  - **Sales**: Product sales (in thousands of units)

## **Tasks**

Based on this data and your final regression model, answer the following questions:

1. **Relationship Between Advertising Budget and Sales**
   - Is there a statistically significant relationship between the advertising budget and sales?

2. **Contribution of Each Media**
   - Do all three media channels—TV, Radio, and Newspaper—contribute to sales?
   - Which media have significant effects on sales?

3. **Media Generating the Biggest Boost in Sales**
   - Which advertising medium generates the largest increase in sales per unit increase in budget?

4. **Strength of the Relationship**
   - How strong is the relationship between the advertising budget and sales?
   - What is the coefficient of determination (R-squared) of your model?

5. **Effect of TV Advertising**
   - How much increase in sales is associated with a given increase in TV advertising budget?

6. **Effect of Radio Advertising**
   - How much increase in sales is associated with a given increase in Radio advertising budget?

7. **Accuracy of Estimated Effects**
   - How accurately can we estimate the effect of each medium on sales?
   - Provide the confidence intervals for the coefficients of each medium.

8. **Predicting Future Sales**
   - How accurately can we predict future sales based on the advertising budgets?
   - What is the standard error of the estimate?

9. **Optimal Allocation of Advertising Budget**
    - Imagine you have a budget of $100,000. What is the best strategy to allocate this budget among TV, Radio, and Newspaper advertising to maximize sales?

10. **Predicting Sales for Specific Budget Allocation**
    - If you spend $10,000 on TV advertising and $20,000 on Radio advertising, how much increase in sales can you expect?

11. **Confidence Interval for Predicted Sales**
    - What is the 95% confidence interval for the predicted sales in the previous question?

12. **Checking Correlation Between Independent Variables**
    - Are there significant correlations between the advertising budgets for different media?
    - How might multicollinearity affect your regression model?




In [ ]:
# Load the data
data = pd.read_csv("https://raw.githubusercontent.com/francji1/01RAD/main/data/Advert.csv", sep=",")
data.head()

In [ ]:
from pandas.plotting import scatter_matrix

# Drop the index column if necessary
data = data.drop(columns=['Unnamed: 0'], errors='ignore')

# Generate scatter matrix
scatter_matrix(data, figsize=(10, 10), diagonal='kde', alpha=0.7, marker='o')
plt.suptitle("Scatter Matrix of Advertising Data")
plt.show()

## Task 1
**Is there a statistically significant relationship between the advertising budget and sales?**

In [ ]:
# Define the predictors and response variable
X = data[['TV', 'radio', 'newspaper']]
y = data['sales']

# Add a constant term for the intercept
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary
print(model.summary())

## Task 1
There is a statistical significant relationship as the F-statistic is 570 with pvalue of 1.59e-96.



## Task 2
- **Do all three media channels—TV, Radio, and Newspaper—contribute to sales?**
- **Which media have significant effects on sales?**


Additionally, from the output we can see that not all media contributes equally to sales. TV and radio have significat relationship with pvalues < 0.05, while newspaper has a pvalue of 0.860 and may not contribute that much.


In [ ]:
# Simple linear regression models
model_tv = smf.ols('sales ~ TV', data=data).fit()
print(model_tv.summary())

model_ra = smf.ols('sales ~ radio', data=data).fit()
print(model_ra.summary())

model_np = smf.ols('sales ~ newspaper', data=data).fit()
print(model_np.summary())

In [ ]:
# Predictions for simple models
new_data = pd.DataFrame({
    'TV': np.arange(0, 301, 5),
    'radio': np.arange(0, 301, 5),
    'newspaper': np.arange(0, 301, 5)
})

predictions_tv = model_tv.get_prediction(new_data).summary_frame()
predictions_ra = model_ra.get_prediction(new_data).summary_frame()
predictions_np = model_np.get_prediction(new_data).summary_frame()

# Plotting
sns.scatterplot(x='TV', y='sales', data=data)
sns.lineplot(x=new_data['TV'], y=predictions_tv['mean'], color="red")
plt.fill_between(new_data['TV'], predictions_tv['obs_ci_lower'], predictions_tv['obs_ci_upper'], color='blue', alpha=0.3)
plt.show()

In [ ]:
# Create a figure with subplots
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# TV vs Sales
sns.scatterplot(x='TV', y='sales', data=data, ax=axs[0])
model_tv = smf.ols('sales ~ TV', data=data).fit()
sns.lineplot(x='TV', y=model_tv.predict(data['TV']), data=data, ax=axs[0], color='blue')

# Radio vs Sales
sns.scatterplot(x='radio', y='sales', data=data, ax=axs[1])
model_ra = smf.ols('sales ~ radio', data=data).fit()
sns.lineplot(x='radio', y=model_ra.predict(data['radio']), data=data, ax=axs[1], color='blue')

# Newspaper vs Sales
sns.scatterplot(x='newspaper', y='sales', data=data, ax=axs[2])
model_np = smf.ols('sales ~ newspaper', data=data).fit()
sns.lineplot(x='newspaper', y=model_np.predict(data['newspaper']), data=data, ax=axs[2], color='blue')

# Set the title for each subplot
axs[0].set_title('TV Advertisements vs Sales')
axs[1].set_title('Radio Advertisements vs Sales')
axs[2].set_title('Newspaper Advertisements vs Sales')

plt.tight_layout()
plt.show()


## Task 3
**Which advertising medium generates the largest increase in sales per unit increase in budget?**

Radio generates the largest increase in sales with a coefficient of 0.1885. TV has a coefficient of 0.0458.

## Task 4
- **How strong is the relationship between the advertising budget and sales?**
- **What is the coefficient of determination (R-squared) of your model?**

It is suggested that the relationship is strong with an R squared of 0.897.



In [ ]:
# Models with and without interactions
model0 = smf.ols('sales ~ TV * radio * newspaper', data=data).fit()
print(model0.summary())
print(model0.conf_int())

model1 = smf.ols('sales ~ TV + radio + newspaper', data=data).fit()
print(model1.summary())
print(model1.conf_int())

model2 = smf.ols('sales ~ TV * radio', data=data).fit()
print(model2.summary())
print(model2.conf_int())

## Task 5
**How much increase in sales is associated with a given increase in TV advertising budget?**


For every additional thousand dollars spent on TV advertising, sales increase by circa 45.8 units.

## Task 6
**How much increase in sales is associated with a given increase in Radio advertising budget?**

189 units.

## Task 7
- **How accurately can we estimate the effect of each medium on sales?**
- **Provide the confidence intervals for the coefficients of each medium.**


If we do not want to change the confidence level, these intervals can be read from the summary of the model on the right side below: [0.025,0.975]

In [ ]:
conf_int = model.conf_int(alpha=0.05) # Can adjust the confidence with alpha
conf_int.columns = ['Lower Bound', 'Upper Bound']
print(conf_int)

## Task 8
- **How accurately can we predict future sales based on the advertising budgets?**
- **What is the standard error of the estimate?**

We can look at the Root Mean Squared Error or Standard Error of Estimate.These values are not directly in the model summary, we have to calculate them:

In [ ]:
# Calculate the Residual Sum of Squares (RSS)
RSS = np.sum(model.resid ** 2)

# Degrees of Freedom Residual
df_resid = model.df_resid

# Calculate the Standard Error of the Estimate
std_err_estimate = np.sqrt(RSS / df_resid)
print(f"Standard Error of the Estimate: {std_err_estimate}")

This low SEE coupeled with the high R-squared value suggests that the model predicts future values with reasonable accuracy.

## Task 9
**Imagine you have a budget of $100,000. What is the best strategy to allocate this budget among TV, Radio, and Newspaper advertising to maximize sales?**

We should allocate more budget into media with higher coefficients as they have a greater impact on sales. Consequently, we would focus more on TV and Radio as we have shown that newspaper is not statistically significant.
#### Suggestion
As the coefficients are 0.1885 and 0.0458, their ratio is 4:1 so we could allocate 80k to radio and 20k to TV?

## Task 10
**If you spend \$10,000 on TV advertising and \$20,000 on Radio advertising, how much increase in sales can you expect?**


For prediction we use the regression eq.

In [ ]:
# Coefficients from the model
intercept = model.params['const']
tv_coef = model.params['TV']
radio_coef = model.params['radio']

# Budgets in thousands of dollars
tv_budget = 10  # \$10,000
radio_budget = 20  # \$20,000

# Predicted sales
predicted_sales = intercept + tv_coef * tv_budget + radio_coef * radio_budget
print(f"Predicted Sales: {predicted_sales * 1000} units")

## Task 11
**What is the 95% confidence interval for the predicted sales in the previous question?**


Below, a new dataframe is created, why? Easily fed into our model.

In [ ]:
# Create a new DataFrame with the input values, this is done solely for the purpose
new_data = pd.DataFrame({'const': 1, 'TV': [10], 'radio': [20], 'newspaper': [0]})

# Get the prediction and confidence intervals
predictions = model.get_prediction(new_data)
prediction_summary = predictions.summary_frame(alpha=0.05) # + confidence specifier

print(prediction_summary[['mean', 'mean_ci_lower', 'mean_ci_upper']])


We have a 95% confidence interval [6.619784,7.714489] for sales. The output here is in thousands of units.


## Task 12
- **Are there significant correlations between the advertising budgets for different media?**
- **How might multicollinearity affect your regression model?**

In [ ]:
# Correlation matrix
corr_matrix = data[['TV', 'radio', 'newspaper']].corr()
print(corr_matrix)

From the above correlation matrix we can infer the following:
- there is a very weak correlation between Tv and newspaper and between Tv and radio
- there is a moderate correlation between radio and newspaper

We could test the significance of the correlations:

In [ ]:
# Function to calculate p-values for correlations
def correlation_test(x, y):
    corr_coef, p_value = stats.pearsonr(x, y)
    return corr_coef, p_value

# TV and Radio sig.
corr_tv_radio, p_tv_radio = correlation_test(data['TV'], data['radio'])
print(f"TV and Radio Correlation: {corr_tv_radio}, p-value: {p_tv_radio}")

# TV and Newspaper sig.
corr_tv_news, p_tv_news = correlation_test(data['TV'], data['newspaper'])
print(f"TV and Newspaper Correlation: {corr_tv_news}, p-value: {p_tv_news}")

# Radio and Newspaper sig.
corr_radio_news, p_radio_news = correlation_test(data['radio'], data['newspaper'])
print(f"Radio and Newspaper Correlation: {corr_radio_news}, p-value: {p_radio_news}")

- Correlation between Tv and radio is not statistically significant.
- Correlation between Tv and newspaper is not statistically significant.
- Correlation between radio and newspaper **is** statistically significant with pvalue of 2.32e-07, which is way below 0.05.

Because of this, we could now test for multicollinearity issues in our model.

Here I follow AI - usage of **VIF** = quantifying how much the variance of a regression coefficient is inflated due to multicollinearity.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Define the independent variables (exclude the dependent variable 'sales')
X_variables = data[['TV', 'radio', 'newspaper']]

# Add a constant term for intercept
X_with_constant = sm.add_constant(X_variables)

# Calculate VIF for each variable
vif_data = pd.DataFrame()
vif_data["feature"] = X_with_constant.columns
vif_data["VIF"] = [variance_inflation_factor(X_with_constant.values, i) for i in range(X_with_constant.shape[1])]

print(vif_data)

In [ ]:
data.columns[0:-1]

In [ ]:
variables = data.columns[0:-1]
vif_dict = {}
df_x = data.iloc[:, 0:-1]
print(variables)
for variable in variables:
    # The independent variables set.
    x_vars = df_x.drop([variable], axis=1)
    # The dependent variable.
    y_var = df_x[variable]

    # Add constant for OLS model
    x_vars_const = sm.add_constant(x_vars)
    # Fit the model
    model = sm.OLS(y_var, x_vars_const).fit()

    # Calculate R-squared value
    rsq = model.rsquared
    #print(model.summary())
    # Calculate VIF
    vif = 1 / (1 - rsq)

    vif_dict[variable] = vif

# Display the VIF values
vif_dict


We can see that the VIF values for the predictors are very low(we do not really care about VIF for the constant term). Therefore, we can conclude that multicollinearity is not a concern among our independent variables.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Scatter plot
sns.pairplot(data[['TV', 'radio', 'newspaper']])
plt.show()

# Without newspaper

In [ ]:
# Define predictors excluding 'newspaper'
X_no_newspaper = data[['TV', 'radio']]
X_no_newspaper = sm.add_constant(X_no_newspaper)

# Fit the model without 'newspaper'
model_no_newspaper = sm.OLS(y, X_no_newspaper).fit()

# Print the summary
print(model_no_newspaper.summary())


- We can see that the R-squared is the same as before - the explanatory power of the model is not reduced, suggesting that newspaper category does not contribute significantly

- The coefficients for TV and Radio remain unchagned, which further proves their significant relationship with sales.
- There is a sligh decrease in standard errors -> showing improved precision after removing newspaper
- The F-statistic increased to 859 from 570 -> improvement

